In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import os
import csv

In [2]:
mortality_with_sundown_census_poverty = pd.read_csv('../data/mortality_with_sundown_census_poverty.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mortality_with_sundown_census_poverty = mortality_with_sundown_census_poverty.rename(
    columns={
        'All Ages in Poverty Percent': 'perc_poverty',
        'Median Household Income in Dollars': 'median_income_str'
    })

In [4]:
df_sundown = pd.read_csv('../data/sundown_with_counties.csv')
analysis_df = pd.merge(mortality_with_sundown_census_poverty, df_sundown, how='left', left_on='County', right_on="county")

In [5]:
# pull out state
analysis_df['state'] = analysis_df.County.apply(lambda x: x.split(', ')[1])

In [6]:
# merge in state shootings data
state_shootings = pd.read_csv('../data/police-killings-state.csv')
analysis_df = pd.merge(analysis_df, state_shootings, left_on='state', right_on='State Abbreviation', how='left')

In [7]:
# layer in jim crow states
jim_crow_states = [
    'TX', 'OK', 'MO', 'AR', 'LA', 'KY', 'WV', 'MD', 'VA', 'TN', 'NC', 'MS', 'AL', 'GA', 'SC', 'FL'
]
analysis_df['is_jim_crow_state'] = analysis_df.state.apply(lambda x: x in jim_crow_states)

In [8]:
# layer in state racism index
racism_index = pd.read_csv('../data/racism_index.csv')
analysis_df = pd.merge(analysis_df, racism_index, left_on='state', right_on='state_abbr', how='left')

In [9]:
# Merge in city shootings df
city_shootings = pd.read_csv('../data/police-killings-city.csv')
analysis_df = pd.merge(analysis_df, city_shootings, left_on='County', right_on='county', how='left')

In [81]:
analysis_df_with_city_shootings = analysis_df.dropna(subset=['Killings by Police per 10k Arrests'])

analysis_df_with_city_shootings['killings_per_10k'] = analysis_df_with_city_shootings['Killings by Police per 10k Arrests']
analysis_df_with_city_shootings['diss_index'] = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
analysis_df_with_city_shootings['num_sundown_towns'] = analysis_df_with_city_shootings['#']
analysis_df_with_city_shootings['perc_white'] = 1 - analysis_df_with_city_shootings['perc_minority']

# coerce White, Black to int
analysis_df_with_city_shootings['White_pop'] = analysis_df_with_city_shootings['White'].apply(lambda x: int(x.replace(',', '')))
analysis_df_with_city_shootings['Black_pop'] = analysis_df_with_city_shootings['Black'].apply(lambda x: int(x.replace(',', '')))
analysis_df_with_city_shootings['black_to_white_shootings_ratio'] = (analysis_df_with_city_shootings['White People Killed by Police (1/1/2013-12/31/2019)'] / analysis_df_with_city_shootings['White_pop']) / (analysis_df_with_city_shootings['Black People Killed by Police (1/1/2013-12/31/2019)'] / analysis_df_with_city_shootings['Black_pop'])

analysis_df_with_city_shootings = analysis_df_with_city_shootings.dropna(subset=['black_to_white_shootings_ratio'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [82]:
analysis_df['median_income'] = analysis_df.median_income_str.apply(lambda x: float(x.replace('$', '').replace(',', ''))/10000)
analysis_df_with_city_shootings['median_income'] = analysis_df_with_city_shootings.median_income_str.apply(lambda x: float(x.replace('$', '').replace(',', ''))/10000)

# Analysis using full dataframe (full analysis_df)

In [50]:
# analysis using full dataframe

y = analysis_df['Death_Population_Ratio']
logit_covariates = [
    'perc_minority',
    '#',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]


In [51]:
logit_model = sm.Logit(y.astype(float), X.astype(float))
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.014563
         Iterations 9


In [52]:
result.summary2()

/usr/local/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:3390: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:              Logit                  Pseudo R-squared: inf       
Dependent Variable: Death_Population_Ratio AIC:              11492.1915
Date:               2020-11-16 20:28       BIC:              11546.6147
No. Observations:   394212                 Log-Likelihood:   -5741.1   
Df Model:           4                      LL-Null:          0.0000    
Df Residuals:       394207                 LLR p-value:      1.0000    
Converged:          1.0000                 Scale:            1.0000    
No. Iterations:     9.0000                                             
------------------------------------------------------------------------
                    Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------------
perc_minority       1.0630    0.1082    9.8211  0.0000   0.8509   1.2752
#                   0.0076    0.0026    2.9072  0.0036   0.0025   0.0127
perc_poverty       -0.1137    0.0024  -46.4366  0.0000  -0.1185  -0.1089
is_jim_crow_state  -0.1988    0.0349   -5.6964  0.0000  -0.2671  -0.1304
median_income      -0.4836    0.0051  -94.8202  0.0000  -0.4936  -0.4736
=======================================================================

"""

# Analysis using just cities we have shootings data for

1. regress (minority%, # of sundown town, perc of poverty, jim crow state, median income) -> killings by police

In [60]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    '#',
    'perc_poverty',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()


In [61]:
ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()

In [62]:
ols_result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                Results: Ordinary least squares
===============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.441      
Dependent Variable:     black_to_white_shootings_ratio AIC:                         278631.5694
Date:                   2020-11-16 20:31               BIC:                         278677.8755
No. Observations:       77748                          Log-Likelihood:              -1.3931e+05
Df Model:               5                              F-statistic:                 1.225e+04  
Df Residuals:           77743                          Prob (F-statistic):          0.00       
R-squared (uncentered): 0.441                          Scale:                       2.1082     
----------------------------------------------------------------------------------------------------
                           Coef.       Std.Err.          t          P>|t|        [0.025       0.975]
----------------------------------------------------------------------------------------------------
perc_minority             -7.0333        0.0587      -119.8460      0.0000      -7.1483      -6.9183
#                         -0.0299        0.0006       -48.8427      0.0000      -0.0311      -0.0287
perc_poverty               0.0802        0.0012        68.5921      0.0000       0.0779       0.0825
is_jim_crow_state         -0.3712        0.0153       -24.2303      0.0000      -0.4013      -0.3412
median_income              0.3325        0.0023       145.1978      0.0000       0.3280       0.3370
-----------------------------------------------------------------------------------------------
Omnibus:                     56130.776               Durbin-Watson:                  0.214     
Prob(Omnibus):               0.000                   Jarque-Bera (JB):               918532.526
Skew:                        3.395                   Prob(JB):                       0.000     
Kurtosis:                    18.409                  Condition No.:                  221       
===============================================================================================

"""

2.  Regression with JIM Crow + Sundowntowns + % minority + median_income + perc_poverty → Black White Dissimilarity Index (2010)

In [63]:
y = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
logit_covariates = [
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Ordinary least squares
=======================================================================================================
Model:                  OLS                                    Adj. R-squared (uncentered): 0.645      
Dependent Variable:     Black-White Dissimilarity Index (2010) AIC:                         774706.9453
Date:                   2020-11-16 20:31                       BIC:                         774725.4677
No. Observations:       77748                                  Log-Likelihood:              -3.8735e+05
Df Model:               2                                      F-statistic:                 7.062e+04  
Df Residuals:           77746                                  Prob (F-statistic):          0.00       
R-squared (uncentered): 0.645                                  Scale:                       1244.4     
--------------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.           t            P>|t|          [0.025         0.975]
--------------------------------------------------------------------------------------------------------------
#                           3.1268          0.0093        337.1340        0.0000         3.1086         3.1450
is_jim_crow_state          45.0082          0.3283        137.0929        0.0000        44.3648        45.6517
-------------------------------------------------------------------------------------------------------
Omnibus:                          7901.686                  Durbin-Watson:                     0.093   
Prob(Omnibus):                    0.000                     Jarque-Bera (JB):                  6881.886
Skew:                             0.651                     Prob(JB):                          0.000   
Kurtosis:                         2.346                     Condition No.:                     36      
=======================================================================================================

"""

In [64]:
y = analysis_df_with_city_shootings['Black-White Dissimilarity Index (2010)']
logit_covariates = [
    '#',
    'is_jim_crow_state',
    'median_income',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Results: Ordinary least squares
=======================================================================================================
Model:                  OLS                                    Adj. R-squared (uncentered): 0.832      
Dependent Variable:     Black-White Dissimilarity Index (2010) AIC:                         716451.4719
Date:                   2020-11-16 20:31                       BIC:                         716479.2556
No. Observations:       77748                                  Log-Likelihood:              -3.5822e+05
Df Model:               3                                      F-statistic:                 1.285e+05  
Df Residuals:           77745                                  Prob (F-statistic):          0.00       
R-squared (uncentered): 0.832                                  Scale:                       588.24     
--------------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.           t            P>|t|          [0.025         0.975]
--------------------------------------------------------------------------------------------------------------
#                           0.9116          0.0099         92.4451        0.0000         0.8923         0.9309
is_jim_crow_state          13.1118          0.2504         52.3717        0.0000        12.6211        13.6025
median_income               6.3932          0.0217        294.4960        0.0000         6.3507         6.4358
-------------------------------------------------------------------------------------------------------
Omnibus:                          3399.171                  Durbin-Watson:                     0.197   
Prob(Omnibus):                    0.000                     Jarque-Bera (JB):                  3092.288
Skew:                             -0.431                    Prob(JB):                          0.000   
Kurtosis:                         2.540                     Condition No.:                     42      
=======================================================================================================

"""

3. regress (minority%, # of sundown town, sundown county status, perc of poverty, jim crow state, median income) -> ratio of blacks / whites killings by police

In [83]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Ordinary least squares
==============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.401     
Dependent Variable:     black_to_white_shootings_ratio AIC:                         23613.3938
Date:                   2020-11-17 22:09               BIC:                         23641.1775
No. Observations:       77748                          Log-Likelihood:              -11804.   
Df Model:               3                              F-statistic:                 1.734e+04 
Df Residuals:           77745                          Prob (F-statistic):          0.00      
R-squared (uncentered): 0.401                          Scale:                       0.079325  
---------------------------------------------------------------------------------------------------
                           Coef.       Std.Err.         t          P>|t|        [0.025       0.975]
---------------------------------------------------------------------------------------------------
perc_minority              0.7332        0.0057      127.6636      0.0000       0.7219       0.7444
#                         -0.0011        0.0001       -9.7739      0.0000      -0.0013      -0.0009
is_jim_crow_state          0.1700        0.0029       58.6934      0.0000       0.1643       0.1757
----------------------------------------------------------------------------------------------
Omnibus:                      53585.664              Durbin-Watson:                 0.055     
Prob(Omnibus):                0.000                  Jarque-Bera (JB):              797937.542
Skew:                         3.210                  Prob(JB):                      0.000     
Kurtosis:                     17.321                 Condition No.:                 80        
==============================================================================================

"""

4. Add interaction term: white_perc * diss_index

In [84]:
y = analysis_df_with_city_shootings['black_to_white_shootings_ratio']
logit_covariates = [
    'perc_minority',
    'median_income',
    '#',
    'is_jim_crow_state',
    'Black-White Dissimilarity Index (2010)',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # This is added back by InteractiveShellApp.init_path()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Results: Ordinary least squares
==============================================================================================
Model:                  OLS                            Adj. R-squared (uncentered): 0.474     
Dependent Variable:     black_to_white_shootings_ratio AIC:                         13506.4969
Date:                   2020-11-17 22:09               BIC:                         13552.8030
No. Observations:       77748                          Log-Likelihood:              -6748.2   
Df Model:               5                              F-statistic:                 1.401e+04 
Df Residuals:           77743                          Prob (F-statistic):          0.00      
R-squared (uncentered): 0.474                          Scale:                       0.069654  
-----------------------------------------------------------------------------------------------
                                           Coef.   Std.Err.     t      P>|t|    [0.025   0.975]
-----------------------------------------------------------------------------------------------
perc_minority                             -0.1627    0.0117  -13.8928  0.0000  -0.1857  -0.1398
median_income                              0.0373    0.0004   95.3097  0.0000   0.0365   0.0380
#                                         -0.0049    0.0001  -43.6769  0.0000  -0.0052  -0.0047
is_jim_crow_state                          0.1170    0.0028   42.2055  0.0000   0.1116   0.1224
Black-White Dissimilarity Index (2010)     0.0013    0.0001   25.2715  0.0000   0.0012   0.0014
----------------------------------------------------------------------------------------------
Omnibus:                      52326.307              Durbin-Watson:                 0.055     
Prob(Omnibus):                0.000                  Jarque-Bera (JB):              734111.962
Skew:                         3.127                  Prob(JB):                      0.000     
Kurtosis:                     16.693                 Condition No.:                 747       
==============================================================================================

"""

5. Regression with JIM + Sundown towns + **percentage white+ Black White Dissimilarity Index (2010) → Racism index (2013-2017) in state racism index 2013-2017 file

In [77]:
y = analysis_df_with_city_shootings['racism_index']
logit_covariates = [
    'perc_minority',
    'median_income',
    '#',
    'is_jim_crow_state',
]
X = analysis_df_with_city_shootings[logit_covariates]
X = X.convert_objects(convert_numeric=True)
y = y.convert_objects(convert_numeric=True)

ols_model = sm.OLS(y.astype(float), X.astype(float))
ols_result = ols_model.fit()
ols_result.summary2()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Ordinary least squares
=================================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.956      
Dependent Variable:     racism_index     AIC:                         604150.6515
Date:                   2020-11-16 20:39 BIC:                         604187.6964
No. Observations:       77748            Log-Likelihood:              -3.0207e+05
Df Model:               4                F-statistic:                 4.268e+05  
Df Residuals:           77744            Prob (F-statistic):          0.00       
R-squared (uncentered): 0.956            Scale:                       138.75     
------------------------------------------------------------------------------------
                      Coef.     Std.Err.       t        P>|t|      [0.025     0.975]
------------------------------------------------------------------------------------
perc_minority        56.5522      0.3922    144.1812    0.0000    55.7834    57.3209
median_income         5.5358      0.0172    321.5183    0.0000     5.5020     5.5695
#                     0.4037      0.0049     82.3186    0.0000     0.3941     0.4133
is_jim_crow_state    -7.7471      0.1227    -63.1554    0.0000    -7.9875    -7.5067
---------------------------------------------------------------------------------
Omnibus:                   965.406           Durbin-Watson:              0.008   
Prob(Omnibus):             0.000             Jarque-Bera (JB):           1515.343
Skew:                      0.120             Prob(JB):                   0.000   
Kurtosis:                  3.640             Condition No.:              136     
=================================================================================

"""